<a href="https://colab.research.google.com/github/ashish1610dhiman/CSE8803_DLT_Project/blob/main/2_model_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [88]:
# !pip install nflows

In [89]:
import pickle
import torch
from torch.utils.data import Dataset, DataLoader
from google.colab import drive
import torch
import torch.nn as nn
from nflows.flows.base import Flow
from nflows.distributions import StandardNormal
from nflows.transforms import CompositeTransform, ReversePermutation, MaskedAffineAutoregressiveTransform
from nflows.nn.nets import ResidualNet
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [90]:
#exp params
VERSION = "v0" #meta for saving
drive.mount('/content/drive')
EXP_PATH = '/content/drive/My Drive/call_prices_conditional_flow/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


0. Load Dataset and create DataLoader

In [91]:
class PackedOptionDataset(Dataset):
    def __init__(self, data):
        self.X = data["X"]
        self.Y = data["Y"]
        self.T = data["T"]
        self.meta = data["meta"]
        self.is_test = data.get("is_test", torch.zeros(len(self.X), dtype=torch.bool))

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return {
            "X": self.X[idx],
            "Y": self.Y[idx],
            "T": self.T[idx],
            "meta": self.meta[idx],
            "is_test": self.is_test[idx]
        }

In [92]:
with open(f"{EXP_PATH}/train_test_val_dataste_{VERSION}.pkl", "rb") as f:
    dataset_splits = pickle.load(f)

In [93]:
#read data dict from google colab
with open(f"{EXP_PATH}/dataset_{VERSION}.pkl", 'rb') as f:
    combined_data_dict = pickle.load(f)

### Dataloaders

In [94]:
dataset_splits["train"]["meta"][0]

tensor([0.0500, 0.1400, 4.0000])

In [95]:
train_set = PackedOptionDataset(dataset_splits["train"])
val_set   = PackedOptionDataset(dataset_splits["val"])
test_set  = PackedOptionDataset(dataset_splits["test"])

In [96]:
BATCH_SIZE = 64  # or any size you prefer

train_loader = DataLoader(
    train_set, batch_size=64, shuffle=True, num_workers=2, pin_memory=True
)
val_loader = DataLoader(
    val_set, batch_size=64, shuffle=False, num_workers=2, pin_memory=True
)
test_loader  = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False)

print(f"Train size: {len(train_loader.dataset)} | Val: {len(val_loader.dataset)} | Test: {len(test_loader.dataset)}")

Train size: 479294 | Val: 102706 | Test: 105000


# 1. Create & train model

In [97]:
# --- Embedding network ---
class EmbeddingNet(nn.Module):
    def __init__(self, input_dim, embed_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, embed_dim)
        )

    def forward(self, x):
        return self.net(x)

In [98]:
def create_conditional_flow(y_dim, context_dim, hidden_dim=64, num_blocks=4):
    transforms = []

    for _ in range(num_blocks):
        transforms.append(
            MaskedAffineAutoregressiveTransform(
                features=y_dim,
                hidden_features=hidden_dim,
                context_features=context_dim,
                num_blocks=2,
                use_residual_blocks=True,
                activation=torch.relu,
                dropout_probability=0.0,
                random_mask=False
            )
        )
        transforms.append(ReversePermutation(features=y_dim))

    transform = CompositeTransform(transforms)
    base_dist = StandardNormal([y_dim])

    return Flow(transform=transform, distribution=base_dist)

In [99]:
class EmbeddingFlowModel(nn.Module):
    def __init__(self, x_dim, y_dim, embed_dim=32, flow_hidden=64, flow_blocks=4):
        super().__init__()
        self.embedding_net = EmbeddingNet(input_dim=x_dim, embed_dim=embed_dim)
        self.flow = create_conditional_flow(
            y_dim=y_dim, context_dim=embed_dim,
            hidden_dim=flow_hidden, num_blocks=flow_blocks
        )

    def forward(self, x, y):
        context = self.embedding_net(x)
        log_prob = self.flow.log_prob(inputs=y, context=context)
        return log_prob

    def sample(self, x, num_samples=1):
        context = self.embedding_net(x)
        return self.flow.sample(num_samples=num_samples, context=context)

In [100]:
# --- Initialize model ---
x_dim = train_set.X.shape[1]
y_dim = train_set.Y.shape[1]

model = EmbeddingFlowModel(x_dim=x_dim, y_dim=y_dim)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
n_epochs = 10
log_interval = 1

In [101]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [ ]:
for epoch in range(1, n_epochs + 1):
    #Training
    model.train()
    train_loss = 0
    for batch in train_loader:
        x = batch["X"].to("cuda")
        y = batch["Y"].to("cuda")
        log_prob = model(x, y)
        loss = -log_prob.mean()
        optimizer.zero_grad()
        loss.backward()
        # torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        train_loss += loss.item() * x.size(0)
    train_loss /= len(train_loader.dataset)

    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in val_loader:
            x = batch["X"].to("cuda")
            y = batch["Y"].to("cuda")
            log_prob = model(x, y)
            val_loss += (-log_prob.mean().item()) * x.size(0)

    val_loss /= len(val_loader.dataset)
    print(f"Epoch {epoch:02d} | Train NLL: {train_loss:.4f} | Val NLL: {val_loss:.4f}")

# 3. Validation set metrics

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

model.eval()
all_y_true, all_y_pred = [], []

with torch.no_grad():
    for batch in val_loader:
        x = batch["X"].to(device, non_blocking=True)
        y_true = batch["Y"].to(device, non_blocking=True)

        x_embed = model.embedding_net(x)

        # Use shape from target
        batch_size, y_dim = y_true.shape
        base_mean = torch.zeros((batch_size, y_dim), device=device)

        # 🔧 Unpack tuple
        y_pred = model.flow.sample(num_samples=1, context=x_embed)  # shape: (1, B, y_dim)
        y_pred = y_pred.squeeze(0)  # remove sample dim


        all_y_true.append(y_true.cpu())
        all_y_pred.append(y_pred.cpu())


In [ ]:
# Stack and compute metrics
y_true_full = torch.cat(all_y_true).numpy()
y_pred_full = torch.cat(all_y_pred).numpy()

mse = mean_squared_error(y_true_full, y_pred_full)
mae = mean_absolute_error(y_true_full, y_pred_full)

print(f"📊 Validation MSE: {mse:.4f} | MAE: {mae:.4f}")


In [ ]:
len(val_loader)

In [ ]:
len(val_loader.dataset)

In [ ]:
102706/64

In [ ]:
val_loader

In [ ]:
len(all_y_pred)

In [ ]:
val_set.meta

In [ ]:
BURNIN_WINDOW=50

In [ ]:
# Assume one prediction
idx = 1
y_pred = y_pred_full[idx]        # shape (M,)
print(y_pred)
meta   = val_set.meta[idx]       # (mu, sigma, path_id)
t_idx  = val_set.T[idx].item()   # time index in call_prices

mu, sigma, path_id = meta.detach().cpu().numpy()
gbm_path = combined_data_dict[(0.14, 0.19)]["gbm_paths"]  # shape (n_steps, N_paths)

# Map prediction back to true path in GBM
gbm_t = t_idx + BURNIN_WINDOW
y_true_actual = y_true_full[idx]

In [ ]:
import matplotlib.pyplot as plt

plt.plot(y_true_actual, label="True future spot")
# plt.plot(y_pred, label="Predicted", linestyle="--")
plt.title(f"Path {path_id} | μ={mu}, σ={sigma} | t={gbm_t}")
plt.xlabel("Future time step")
plt.ylabel("Spot price")
plt.legend()
plt.grid(True)
plt.show()